# Микродиахроническое исследование русских приставок методами дистрибутивной семантики
## Автор: Елизавета Клыкова, БКЛ181
### Получение эмбеддингов ELMo для всех глаголов

#### Импорт модулей

In [1]:
# %load_ext pycodestyle_magic
# %pycodestyle_on

In [2]:
!pip install simple_elmo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 179.5 kB/s eta 0:00:00a 0:00:01


In [3]:
import pickle
import random
import pandas as pd
from tqdm.auto import tqdm
from simple_elmo import ElmoModel

import warnings
warnings.filterwarnings('ignore')

#### Получение списка глаголов и предложений

In [4]:
with open('../input/elmo-rnc/presov_sents.pickle', 'rb') as f:
    presov_sents = pickle.load(f)

In [5]:
with open('../input/elmo-rnc/sov_sents.pickle', 'rb') as f:
    sov_sents = pickle.load(f)

In [6]:
with open('../input/elmo-rnc/postsov_sents.pickle', 'rb') as f:
    postsov_sents = pickle.load(f)

In [7]:
pref_df = pd.read_csv('../input/elmo-rnc/prefixes_and_lemmas_dropna.tsv', sep='\t')
pref_df

,prefix,lemma,abs_freq,abs_freq0,abs_freq1,abs_freq2,freq,freq0,freq1,freq2
0,бе[зс],бездействовать,541,155,255,131,2.194,2.147,2.741,1.609
1,бе[зс],бездельничать,308,41,169,98,1.249,0.568,1.817,1.204
2,бе[зс],безмолвствовать,726,361,191,174,2.944,5.000,2.053,2.138
3,бе[зс],безобразить,149,99,43,7,0.604,1.371,0.462,0.086
4,бе[зс],безобразничать,306,110,136,60,1.241,1.524,1.462,0.737
...,...,...,...,...,...,...,...,...,...,...
8429,у,ущемляться,62,8,16,38,0.251,0.111,0.172,0.467
8430,у,ущипывать,510,147,222,141,2.068,2.036,2.386,1.732
8431,у,уязвлять,1305,606,411,288,5.291,8.393,4.418,3.538
8432,у,уяснять,1803,677,740,386,7.311,9.377,7.954,4.742


In [8]:
# list of dicts
pref_dict = pref_df.to_dict(orient='records')

In [9]:
prefixes = set([row['prefix'] for row in pref_dict])

In [10]:
prefs_with_verbs = {pref: [] for pref in prefixes}

for row in pref_dict:
    prefs_with_verbs[row['prefix']].append(row)

In [11]:
prefs_verbs_10 = {pref: [] for pref in prefixes}
for pref in prefs_with_verbs:
    prefs_verbs_10[pref].extend([v['lemma'] for v in sorted(prefs_with_verbs[pref],
                                                            key=lambda x:x['freq'],
                                                            reverse=True)[0:10]])

In [12]:
# with open('prefs_verbs_10.pickle', 'wb') as f:
#     pickle.dump(prefs_verbs_10, f)

In [13]:
verbs_to_search = []
for pref in prefs_verbs_10:
    verbs_to_search.extend(prefs_verbs_10[pref])
len(verbs_to_search)

260

In [14]:
# with open('verbs_to_search.pickle', 'wb') as f:
#     pickle.dump(verbs_to_search, f)

In [15]:
presov_samples = {}
sov_samples = {}
postsov_samples = {}

for verb in verbs_to_search:
    presov_samples[verb] = presov_sents[verb][0:10]
    sov_samples[verb] = sov_sents[verb][0:10]
    postsov_samples[verb] = postsov_sents[verb][0:10]

#### Загрузка модели

In [16]:
model = ElmoModel()
model.load('../input/elmo-pretrained/ruwikiruscorpora_tokens_elmo_1024_2019')

2022-05-27 07:38:36.869620: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:36.965560: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:36.966385: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


'The model is now loaded.'

#### Получение эмбеддингов ELMo
(Ryzhova et al. 2021)

For each word we find all the sentences where these words occur and construct 100 random pairs. The first sentence in a pair belongs to one time period, the second to the other period. For each sentence we compute the ELMo embedding of the word for which we want to measure the semantic shift value. The final score is the average cosine similarity for all pairs. To achieve a certain level of robustness, we take the average value of the cosine similarity from 5 experiments on different 100 random pairs in each iteration.

In [18]:
for idx, verb in enumerate(verbs_to_search):

    presov_inputs = [sent[0] for sent in presov_samples[verb]]
    presov_embeds = model.get_elmo_vectors(presov_inputs)
    for i, sent in enumerate(presov_samples[verb]):
        tup_to_list = list(sent)
        target_emb = presov_embeds[i][tup_to_list[1]]
        tup_to_list.append(target_emb)
        presov_samples[verb][i] = tup_to_list

    print(f'Completed run {idx+1} of {len(verbs_to_search)}')

2022-05-27 07:06:49.831808: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:49.833025: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:49.833899: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:49.834641: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:49.835262: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Completed run 1 of 260


2022-05-27 07:06:52.731154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:52.732051: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:52.732598: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:52.733166: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:52.733647: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Completed run 2 of 260


2022-05-27 07:06:55.436314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:55.437109: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:55.437604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:55.438117: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:06:55.438565: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

...

Completed run 258 of 260


2022-05-27 07:18:24.714804: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:18:24.715577: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:18:24.716121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:18:24.716622: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:18:24.717082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Completed run 259 of 260


2022-05-27 07:18:27.439716: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:18:27.440678: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:18:27.441328: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:18:27.441971: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:18:27.442521: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Completed run 260 of 260


In [19]:
with open('presov_samples.pickle', 'wb') as f:
    pickle.dump(presov_samples, f)

In [17]:
for idx, verb in enumerate(verbs_to_search):

    sov_inputs = [sent[0] for sent in sov_samples[verb]]
    sov_embeds = model.get_elmo_vectors(sov_inputs)
    for i, sent in enumerate(sov_samples[verb]):
        tup_to_list = list(sent)
        target_emb = sov_embeds[i][tup_to_list[1]]
        tup_to_list.append(target_emb)
        sov_samples[verb][i] = tup_to_list
    
    print(f'Completed run {idx+1} of {len(verbs_to_search)}')

2022-05-27 07:23:38.437953: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-27 07:23:38.439300: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:23:38.440980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:23:38.442371: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Completed run 1 of 260


2022-05-27 07:23:50.785134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:23:50.786073: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:23:50.786704: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:23:50.787273: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:23:50.787788: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Completed run 2 of 260


2022-05-27 07:23:53.130055: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:23:53.130979: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:23:53.131554: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:23:53.132065: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:23:53.132547: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

...

Completed run 258 of 260


2022-05-27 07:35:01.406696: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:35:01.407622: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:35:01.408224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:35:01.408762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:35:01.409268: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Completed run 259 of 260


2022-05-27 07:35:04.100071: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:35:04.100901: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:35:04.101416: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:35:04.101913: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:35:04.102378: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Completed run 260 of 260


In [19]:
with open('sov_samples.pickle', 'wb') as f:
    pickle.dump(sov_samples, f)

In [17]:
for idx, verb in enumerate(verbs_to_search):

    postsov_inputs = [sent[0] for sent in postsov_samples[verb]]
    postsov_embeds = model.get_elmo_vectors(postsov_inputs)
    for i, sent in enumerate(postsov_samples[verb]):
        tup_to_list = list(sent)
        target_emb = postsov_embeds[i][tup_to_list[1]]
        tup_to_list.append(target_emb)
        postsov_samples[verb][i] = tup_to_list
    
    print(f'Completed run {idx+1} of {len(verbs_to_search)}')

2022-05-27 07:38:39.026310: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-27 07:38:39.026895: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:39.028321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:39.029506: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Completed run 1 of 260


2022-05-27 07:38:49.394231: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:49.395180: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:49.395875: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:49.396577: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:49.397221: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Completed run 2 of 260


2022-05-27 07:38:51.586600: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:51.587420: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:51.587929: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:51.588407: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:38:51.588856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

...

Completed run 258 of 260


2022-05-27 07:49:54.427613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:49:54.428471: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:49:54.429030: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:49:54.429539: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:49:54.430004: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Completed run 259 of 260


2022-05-27 07:49:58.518045: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:49:58.518841: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:49:58.519370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:49:58.519906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-27 07:49:58.520386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Completed run 260 of 260


In [18]:
with open('postsov_samples.pickle', 'wb') as f:
    pickle.dump(postsov_samples, f)